In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
import os
import traceback
import geopandas as gpd
from shapely.geometry import Point

# Github/national_address_database/data/state_county/il_crawford.csv.xz

In [2]:
fips_state = pd.read_csv("../data/fips_state.csv", dtype={'fips': str})
fips_county = pd.read_csv("../data/fips_county.csv", dtype={'fips': str})
fips_state.head()

,fips,state,abbr
0,01,alabama,al
1,02,alaska,ak
2,04,arizona,az
3,05,arkansas,ar
4,06,california,ca


In [3]:
fips_county.head()

,fips,county,state,abbr
0,01000,alabama,alabama,al
1,01001,autauga_county,alabama,al
2,01003,baldwin_county,alabama,al
3,01005,barbour_county,alabama,al
4,01007,bibb_county,alabama,al


In [4]:
fip_il = fips_state.fips[fips_state.abbr == 'il'].iloc[0]
il_county_fips = fips_county[fips_county.fips.str.slice(0,2) == fip_il]
il_county_fips

,fips,county,state,abbr
606,17000,illinois,illinois,il
607,17001,adams_county,illinois,il
608,17003,alexander_county,illinois,il
609,17005,bond_county,illinois,il
610,17007,boone_county,illinois,il
...,...,...,...,...
704,17195,whiteside_county,illinois,il
705,17197,will_county,illinois,il
706,17199,williamson_county,illinois,il
707,17201,winnebago_county,illinois,il


In [22]:
with open('../data/state_county.json', 'r') as file:
    county_file_json = json.load(file)
file.close()

county_file_json = county_file_json[0]['contents']
county_file_df = pd.DataFrame.from_dict(county_file_json)
county_file_df

,type,name
0,file,ak_anchorage_municipality.csv.xz
1,file,ak_fairbanks_north_star_borough.csv.xz
2,file,ak_southeast_fairbanks_census_area.csv.xz
3,file,al_autauga_county.csv.xz
4,file,al_baldwin_county.csv.xz
...,...,...
1797,file,wy_laramie.csv.xz
1798,file,wy_natrona.csv.xz
1799,file,wy_teton.csv.xz
1800,file,wy_uinta.csv.xz


In [27]:
il_files = county_file_df.name[county_file_df.name.str.slice(0,2) == 'il']
il_files

462     il_crawford.csv.xz
463        il_edgar.csv.xz
464    il_vermilion.csv.xz
Name: name, dtype: object

In [18]:
shape_url = "https://www2.census.gov/geo/tiger/TIGER2020PL/LAYER/TABBLOCK/2020/tl_2020_17_tabblock20.zip"
block_df = gpd.read_file(shape_url)

In [32]:
block_df['geoid20'] = block_df.STATEFP20 + block_df.COUNTYFP20 + block_df.TRACTCE20 + block_df.BLOCKCE20
block_df

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,geoid20
0,17,057,953800,1047,170579538001047,Block 1047,G5040,NaN,NaN,NaN,S,5365,0,+40.3948787,-090.1583342,"POLYGON ((-90.15877 40.39479, -90.15876 40.395...",170579538001047
1,17,169,970200,3023,171699702003023,Block 3023,G5040,NaN,NaN,NaN,S,8585,0,+40.1210215,-090.5598714,"POLYGON ((-90.56039 40.12145, -90.55996 40.121...",171699702003023
2,17,103,000100,4063,171030001004063,Block 4063,G5040,NaN,NaN,NaN,S,1298159,0,+41.8298424,-088.9511439,"POLYGON ((-88.96107 41.83323, -88.95948 41.833...",171030001004063
3,17,057,953000,4028,170579530004028,Block 4028,G5040,NaN,NaN,NaN,S,2128215,0,+40.5710452,-090.2872662,"POLYGON ((-90.30324 40.57410, -90.30309 40.574...",170579530004028
4,17,057,953900,3025,170579539003025,Block 3025,G5040,NaN,NaN,NaN,S,0,16309,+40.2499578,-090.3729903,"POLYGON ((-90.37469 40.24967, -90.37434 40.250...",170579539003025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369973,17,031,827902,3018,170318279023018,Block 3018,G5040,NaN,NaN,NaN,S,7251,0,+41.5804352,-087.5348995,"POLYGON ((-87.53520 41.58110, -87.53485 41.581...",170318279023018
369974,17,031,826000,2006,170318260002006,Block 2006,G5040,NaN,NaN,NaN,S,10129,0,+41.6182382,-087.5290192,"POLYGON ((-87.52933 41.61915, -87.52896 41.619...",170318260002006
369975,17,031,825803,2025,170318258032025,Block 2025,G5040,NaN,NaN,NaN,S,7840,0,+41.6179119,-087.5575407,"POLYGON ((-87.55786 41.61859, -87.55766 41.618...",170318258032025
369976,17,031,520600,1024,170315206001024,Block 1024,G5040,NaN,NaN,NaN,S,4504,0,+41.6916564,-087.5395049,"POLYGON ((-87.54011 41.69150, -87.54011 41.691...",170315206001024


In [31]:
df = pd.read_csv("../data/state_county/il_vermilion.csv.xz")
df

,state,county,zip,longitude,latitude,address,id
0,il,vermilion,47928,-87.533526,39.980466,"7650 north il state line road,,il,47928",il_vermilion
1,il,vermilion,47928,-87.533221,39.986298,"8062 il state line road,,il,47928",il_vermilion
2,il,vermilion,47928,-87.533883,39.983668,"7868 il state line road,,il,47928",il_vermilion
3,il,vermilion,47928,-87.532856,39.956394,"3500 west county road 500 north,,il,47928",il_vermilion
4,il,vermilion,47928,-87.532719,39.993829,"8564 il state line road,,il,47928",il_vermilion
5,il,vermilion,47928,-87.532760,39.987867,"8160 il state line road,,il,47928",il_vermilion
6,il,vermilion,47928,-87.533082,39.923091,"3500 west county road 250 north,,il,47928",il_vermilion


In [39]:
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=block_df.crs)
geo_df

,state,county,zip,longitude,latitude,address,id,geometry
0,il,vermilion,47928,-87.533526,39.980466,"7650 north il state line road,,il,47928",il_vermilion,POINT (-87.53353 39.98047)
1,il,vermilion,47928,-87.533221,39.986298,"8062 il state line road,,il,47928",il_vermilion,POINT (-87.53322 39.98630)
2,il,vermilion,47928,-87.533883,39.983668,"7868 il state line road,,il,47928",il_vermilion,POINT (-87.53388 39.98367)
3,il,vermilion,47928,-87.532856,39.956394,"3500 west county road 500 north,,il,47928",il_vermilion,POINT (-87.53286 39.95639)
4,il,vermilion,47928,-87.532719,39.993829,"8564 il state line road,,il,47928",il_vermilion,POINT (-87.53272 39.99383)
5,il,vermilion,47928,-87.532760,39.987867,"8160 il state line road,,il,47928",il_vermilion,POINT (-87.53276 39.98787)
6,il,vermilion,47928,-87.533082,39.923091,"3500 west county road 250 north,,il,47928",il_vermilion,POINT (-87.53308 39.92309)


In [42]:
joined_df = block_df.sjoin(geo_df, how='inner',predicate='intersects')
joined_df.loc[:, ['state', 'county', 'zip', 'longitude', 'latitude', 'address', 'id','geoid20']]

,state,county,zip,longitude,latitude,address,id,geoid20
101442,il,vermilion,47928,-87.533082,39.923091,"3500 west county road 250 north,,il,47928",il_vermilion,171830111002062
162501,il,vermilion,47928,-87.533221,39.986298,"8062 il state line road,,il,47928",il_vermilion,171830111001012
162501,il,vermilion,47928,-87.532760,39.987867,"8160 il state line road,,il,47928",il_vermilion,171830111001012
162501,il,vermilion,47928,-87.532719,39.993829,"8564 il state line road,,il,47928",il_vermilion,171830111001012
264083,il,vermilion,47928,-87.532856,39.956394,"3500 west county road 500 north,,il,47928",il_vermilion,171830111001044
275089,il,vermilion,47928,-87.533526,39.980466,"7650 north il state line road,,il,47928",il_vermilion,171830111001033
275089,il,vermilion,47928,-87.533883,39.983668,"7868 il state line road,,il,47928",il_vermilion,171830111001033


In [43]:
for file_name in il_files:
    df = pd.read_csv("../data/state_county/" + file_name)
    geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=block_df.crs)
    joined_df = block_df.sjoin(geo_df, how='inner',predicate='intersects')
    joined_df.loc[:, ['state', 'county', 'zip', 'longitude', 'latitude', 'address', 'id','geoid20']].to_csv("../data/state_county/" + file_name)

In [45]:
pd.read_csv("../data/state_county/il_vermilion.csv.xz")

,Unnamed: 0,state,county,zip,longitude,latitude,address,id,geoid20
0,101442,il,vermilion,47928,-87.533082,39.923091,"3500 west county road 250 north,,il,47928",il_vermilion,171830111002062
1,162501,il,vermilion,47928,-87.533221,39.986298,"8062 il state line road,,il,47928",il_vermilion,171830111001012
2,162501,il,vermilion,47928,-87.532760,39.987867,"8160 il state line road,,il,47928",il_vermilion,171830111001012
3,162501,il,vermilion,47928,-87.532719,39.993829,"8564 il state line road,,il,47928",il_vermilion,171830111001012
4,264083,il,vermilion,47928,-87.532856,39.956394,"3500 west county road 500 north,,il,47928",il_vermilion,171830111001044
5,275089,il,vermilion,47928,-87.533526,39.980466,"7650 north il state line road,,il,47928",il_vermilion,171830111001033
6,275089,il,vermilion,47928,-87.533883,39.983668,"7868 il state line road,,il,47928",il_vermilion,171830111001033
